# SAXS Simulation + Fitting Debug Notebook\n\nThis notebook helps debug the SAXS simulation and fitting pipeline used by NanoOrganizer (`pySAXSFitting` + `pyFitting`).\n\nDefault setup matches your requested baseline:\n- Shape: sphere\n- Radius: 100 A (10 nm)\n- Scale: 100\n- Background: 0.1 (constant)\n- q range: 0.0004 to 0.2 A^-1\n- Polydisperse: True\n- sigma_rel: 0.10\n- Porod: False\n- Noise level: small (0.005 by default)\n\nNoise model in adapter: `noise_sigma = noise_level * ptp(intensity_clean)`.

In [ ]:
from pathlib import Path\nimport sys\nimport numpy as np\nimport pandas as pd\nimport plotly.graph_objects as go

In [ ]:
def find_repo_root(start: Path) -> Path:\n    for candidate in [start, *start.parents]:\n        marker = candidate / 'NanoOrganizer' / 'web_app' / 'components' / 'fitting_adapters.py'\n        if marker.exists():\n            return candidate\n    raise RuntimeError('Could not locate repo root from current working directory.')\n\nREPO_ROOT = find_repo_root(Path.cwd())\nWEB_APP_PATH = REPO_ROOT / 'NanoOrganizer' / 'web_app'\nif str(WEB_APP_PATH) not in sys.path:\n    sys.path.insert(0, str(WEB_APP_PATH))\n\nfrom components.fitting_adapters import (\n    PYSAXS_AVAILABLE,\n    PYSAXS_IMPORT_ERROR,\n    PYFITTING_AVAILABLE,\n    PYFITTING_IMPORT_ERROR,\n    simulate_saxs_curve,\n    run_saxs_fit,\n    create_fit_plot_figure,\n)\n\nprint('Repo root:', REPO_ROOT)\nprint('PYSAXS_AVAILABLE:', PYSAXS_AVAILABLE, '|', PYSAXS_IMPORT_ERROR)\nprint('PYFITTING_AVAILABLE:', PYFITTING_AVAILABLE, '|', PYFITTING_IMPORT_ERROR)

In [ ]:
CFG = {\n    'shape': 'sphere',\n    'q_min': 0.0004,\n    'q_max': 0.2,\n    'n_points': 2000,\n    'radius': 100.0,\n    'scale': 100.0,\n    'polydisperse': True,\n    'sigma_rel': 0.10,\n    'use_porod': False,\n    'background_mode': 'constant',\n    'background_const': 0.10,\n    'background_decay_amp': 0.08,\n    'background_decay_q0': 0.035,\n    'background_decay_exp': 3.5,\n    'noise_level': 0.005,\n    'seed': 1234,\n    # set True if you want clean/background channels in dataframe\n    'include_components': False,\n}\nCFG

In [ ]:
sim_df, sim_meta = simulate_saxs_curve(**CFG)\nsim_df.head(), sim_meta

In [ ]:
q = sim_df['q'].to_numpy(dtype=float)\nI = sim_df['intensity'].to_numpy(dtype=float)\n\nfig = go.Figure()\nfig.add_trace(go.Scatter(x=q, y=I, mode='markers', name='Simulated data', marker=dict(size=4, color='black')))\nfig.update_xaxes(type='log', title='q (A^-1)')\nfig.update_yaxes(type='log', title='Intensity (a.u.)')\nfig.update_layout(height=500, title='Simulated SAXS (log-log)')\nfig.show()\n\nprint('noise_level =', sim_meta.get('noise_level'))\nprint('noise_sigma =', sim_meta.get('noise_sigma'))

In [ ]:
initial_overrides = {\n    'radius': 100.0,\n    'sigma_rel': 0.10,\n    'scale': float(np.max(I)),\n    'background': float(np.min(I)),\n}\n\nfit_state = run_saxs_fit(\n    q,\n    I,\n    shape='sphere',\n    shape_label='Sphere',\n    polydisperse=True,\n    use_porod=False,\n    maxiter=3000,\n    x_col_name='q',\n    y_col_name='intensity',\n    initial_overrides=initial_overrides,\n)\n\nfit_state['success'], fit_state['message']

In [ ]:
params = pd.DataFrame([{'parameter': k, 'value': v} for k, v in sorted(fit_state.get('params', {}).items())])\nmetrics = pd.DataFrame([{'metric': k, 'value': v} for k, v in sorted(fit_state.get('metrics', {}).items())])\n\ntruth_vs_fit = pd.DataFrame([\n    {'name': 'radius', 'truth': CFG['radius'], 'fit': fit_state.get('params', {}).get('radius')},\n    {'name': 'sigma_rel', 'truth': CFG['sigma_rel'], 'fit': fit_state.get('params', {}).get('sigma_rel')},\n    {'name': 'scale', 'truth': CFG['scale'], 'fit': fit_state.get('params', {}).get('scale')},\n    {'name': 'background', 'truth': CFG['background_const'], 'fit': fit_state.get('params', {}).get('background')},\n])\n\ndisplay(metrics)\ndisplay(truth_vs_fit)\ndisplay(params.head(20))

In [ ]:
fit_fig = create_fit_plot_figure(fit_state)\nfit_fig.update_xaxes(type='log', title='q (A^-1)')\nfit_fig.update_yaxes(type='log', title='Intensity (a.u.)')\nfit_fig.update_layout(title='Fit overlay (log-log)', height=520)\nfit_fig.show()\n\nresidual = np.asarray(fit_state['y'], dtype=float) - np.asarray(fit_state['y_fit'], dtype=float)\nfig_res = go.Figure()\nfig_res.add_trace(go.Scatter(x=fit_state['x'], y=residual, mode='markers', marker=dict(size=4, color='black')))\nfig_res.add_hline(y=0.0, line_width=1, line_dash='dash', line_color='red')\nfig_res.update_xaxes(type='log', title='q (A^-1)')\nfig_res.update_yaxes(title='Residual')\nfig_res.update_layout(title='Residuals', height=360)\nfig_res.show()

## Notes for debugging\n- Try `include_components=True` in `CFG` to inspect `intensity_clean`, `intensity_shape`, and `intensity_background`.\n- If fit is unstable, reduce `noise_level` or increase `maxiter`.\n- For mismatch diagnosis, print `fit_state['params']` and compare with `sim_meta['model_params']`.